<img 
  class="-inro-notebook-icon-inro-small" 
  style="margin-left: 0; margin-bottom: 15px;"> 
  © Copyright 2018, INRO.
</img>

Please note that this notebook should be run on a fresh copy of the Winnipeg demonstration project; otherwise you may encounter an error.

In [ ]:
%modeller --show_output false

#Data Table API Guide
This notebook provides a brief overview of the Python Data Table API, which is used to access, modify, and analyze the contents of data tables in Emme. Before executing the code below, ensure the Winnipeg Demonstration Project is open in your Emme Desktop environment.
##Contents

<a href="#Listing-existing-data-tables">Listing existing data tables</a>

<a href="#Accessing-and-modifying-an-existing-data-table">Accessing and modifying an existing data table</a>

<a href="#Creating-a-new-data-table">Creating a new data table</a>
- <a href="#New-data-table-from-a-network-table">New data table from a network table</a>
- <a href="#New-data-table-from-a-shape-file">New data table from a shape file</a>

<a href="#Summarizing-a-data-table">Summarizing a data table</a>

<a href="#Joining-data-tables">Joining data tables</a>

<a href="#Exporting-from-data-tables">Exporting from data tables</a>

<a href="#Performing-a-spatial-join-operation">Performing a spatial join operation</a>

<a href="#Performing-a-buffer-(spatial)-operation">Performing a buffer (spatial) operation</a>

<a href="#Mapping-a-data-table">Mapping a data table</a>

<a href="#Performing-SQL-Queries">Performing SQL Queries</a>

<a href="#Creating-a-new-data-table-database">Creating a new data table database</a>

##Listing existing data tables
First, we will import the <i>display</i> module, which unlike the <i>print</i> command, can be used to view Notebook outputs in-line, even when an instance of Emme Modeller is open. We then set variables which point to our currently open instances of emmebank and Desktop:

In [ ]:
from IPython.display import display
my_modeller = inro.modeller.Modeller()
emmebank = my_modeller.emmebank
desktop = my_modeller.desktop

We can now display a list of the data tables currently defined in our project using the __`tables()`__ method:

In [ ]:
project_table_db = desktop.project.data_tables()
for t in project_table_db.tables():
    display(t.name)

u'district names'

u'facility types'

u'winnipeg districts'

u'winnipeg land use'

u'wpglinks'

##Accessing and modifying an existing data table
We will now access an existing data table from the Winnipeg project and list its columns. First, we use the __`table()`__ method to load the _winnipeg districts_ table by name:

In [ ]:
wpg_districts = project_table_db.table("winnipeg districts")
wpg_districts.name

u'winnipeg districts'

Now, the `Table.`__`get_data()`__  method is used to load the data from the table in the _district_data_ variable:

In [ ]:
# data of data table "winnipeg districts"
district_data = wpg_districts.get_data()

Finally, the `Table.`__`attributes()`__ method is used to access the columns in the _winnipeg districts_ data table. We then loop through each attribute (column) to display its name and data type:

In [ ]:
# name and type of each attribute (column)
for a in district_data.attributes():
    display(a.name + " - " + a.atype)

u'SEQNO - REAL'

u'ITEM - STRING'

u'DISTRICT - INTEGER32'

u'NAME - STRING'

u'DESCRIPTIO - STRING'

...

u'XGRAVITY - REAL'

u'YGRAVITY - REAL'

u'XDISPLAY - REAL'

u'YDISPLAY - REAL'

u'geometry - GEOMETRY'

If we want to rename a column in the data table, the `Data.`__`rename_attribute()`__ method is used.
To remove a column in the data table, the `Data.`__`delete_attribute()`__ method is used.
The cell below will rename the `'DESCRIPTIO'` column to `'DESCRIPTION'`,
delete the `'SEQNO'` column,
and then save this change back to the disk using the __`set_data()`__ method:

In [ ]:
# change the name of column "DESCRIPTIO" and write back the data
district_data.rename_attribute("DESCRIPTIO", "DESCRIPTION")
district_data.delete_attribute("SEQNO")
project_table_db.table("winnipeg districts").set_data(district_data)

The rows of a column can be accessed by first looking up the column with the __`attribute()`__ method, and then using the `Attribute.`__`values`__ property to output all of the values of that column, plus the data type. The contents of the `'ITEM'` column consist of :

In [ ]:
# numpy array with values of NAME attribute
ga_groups = district_data.attribute("ITEM")
ga_groups.values

array([u'ga09', u'ga05', u'ga01', u'ga04', u'ga06', u'ga02', u'ga00',
       u'ga03'], dtype=object)

Additional properties of data table columns/attributes can be accessed, such as the name (__`.name`__), data type (__`.atype`__), and in the case of spatial columns, the shape (__`.geometry_type`__):

In [ ]:
district_geom = district_data.attribute("geometry")
display(district_geom.name, district_geom.atype, district_geom.geometry_type)

u'geometry'

u'GEOMETRY'

u'POLYGON'

## Creating a new data table 
### New data table from a network table

A data table can be created using the contents of an existing network table.
Here our intention is to create a data table consisting of node IDs and their coordinates. 
We will start from the _Table of nodes_ network table, remove the unnecessary columns 
(user data and label) and then save the result as a new data table object called `'nodes'`.

In [ ]:
# creating a data table from a worksheet table
ws_path = ["General", "Network", "Table of nodes"]
root_ws_f = desktop.root_worksheet_folder()
table_item = root_ws_f.find_item(ws_path)
node_table = table_item.open()
for i in range(6, 2, -1):
    node_table.delete_column(i)
nodes_dt = node_table.save_as_data_table("nodes", overwrite=True)
node_table.close()

### New data table from a shape file

We can generate a spatial data table using the contents of an existing shape file. To do so, we must first import the _datatable_ module to be able to import the shape file as a data source. 

In [ ]:
import inro.emme.datatable as _dt

In this example we will import the data from the `WinnipegZones.shp` shape file located in the Media folder of the project and output the file contents:

In [ ]:
# import data from a shape file 
import os
project_path = os.path.dirname(desktop.project.path)
ds_file = os.path.join(project_path, "Media/WinnipegZones.shp").replace("\\", "/")
ds = _dt.DataSource(ds_file)
for l in ds.layers():
    display(l.name, l.spatial_reference.name)

u'WinnipegZones'

u'NAD_1983_UTM_Zone_14N'

We then access the 'WinnipegZones' layer from the shape file and display its attributes and associated data types:

In [ ]:
wpg_zone_data = ds.layer('WinnipegZones').get_data()
for a in wpg_zone_data.attributes():
    display(a.name + " - " + a.atype)

u'ZONEID - INTEGER32'

u'DISTRICT - STRING'

u'EMPLOYMENT - STRING'

u'ZONETYPE - STRING'

u'DESCRIPTIO - STRING'

u'geometry - GEOMETRY'

We now create a new data table (or overwrite an existing data table) named 'winnipeg zones', and populate it with the data from the shape file using the __`create_table()`__ method with the 'overwrite' argument:

In [ ]:
# add new data table to the project Data Tables (overwrite if already exists)
project_table_db.create_table("winnipeg zones", wpg_zone_data, overwrite=True)

Table(winnipeg zones)

## Summarizing a data table

It is possible to compute summary statistics on a data table by grouping the values in one or more columns.
The `Data.`__`summarize()`__ method is used to do this, which takes two arguments:
* __`group_by`__ is a sequence of one or more data table column names containing a hierarchy of table pivot points. Data will be grouped by the first item in this list first, followed by the second item, etc.
* __`operations`__ specify the operation/type of analysis as well as the column for which this analysis will be performed. The `operations` argument contains a list of one or more items. Each item in the list contains a pair consisting of the column name followed by the type of operation.

Operation types can be `FIRST`, `SUM`, `AVERAGE`, `MAXIMUM`, `MINIMUM`, or `COUNT`. 

In the example below, we want to find the average speed per number of lanes.
We will start by creating a new network table of links with the following columns: `i`, `j`, `lanes` and `speedau` and we will restrict the table to road network without the connectors.
Then we will save the link table as a data table object called `'links'`.
Finally we will use the __`summarize`__ function, grouping by lanes and  averaging the speed.

In [ ]:
import inro.emme.desktop.worksheet as _worksheet
project = desktop.project
new_link_table = project.new_network_table("LINK")
new_link_table.par("Filter").set("isAuto && not(isConnector)")
column = _worksheet.Column()
column.name = "i"
column.expression = "i"
column.decimals = 0
new_link_table.add_column(1, column)
column.name = "j"
column.expression = "j"
new_link_table.add_column(2, column)
column.name = "lanes"
column.expression = "lanes"
new_link_table.add_column(3, column)
column.decimals = 2
column.name = "speed"
column.expression = "speedau"
new_link_table.add_column(4, column)
links_dt = new_link_table.save_as_data_table("links", overwrite=True)
new_link_table.close()

In [ ]:
links_data = links_dt.get_data()
link_summary = links_data.summarize(['lanes'], [['speed', 'AVERAGE']])
project_table_db.create_table("link_summary", link_summary, overwrite=True)

Table(link_summary)

The following code uses pandas to display the contents of the resulting data table: 

In [ ]:
import pandas as _pd
def to_pandas(dat):
    dfs = []
    for a in dat.attributes():
        if a.atype == 'GEOMETRY':
            dfs.append( _pd.DataFrame( {a.name: [x.text for x in a.values]} ) )
        else:
            dfs.append( _pd.DataFrame( {a.name: a.values} ) )
    df = dfs[0]
    for x in dfs[1:]:
        df = df.join(x)
    return df
to_pandas(link_summary)

,lanes,Avg(speed)
0,2,53.855985
1,1,52.621626
2,3,48.494634
3,4,40.369942


## Joining data tables

In this example, we want to obtain the district-to-district totals of a demand matrix.
We will summarize an O-D Pair network table by district, and then use join operations to add district names to the resulting data table. Let's first create the O-D pair table.

In [ ]:
# create an O-D pair network table wit columns p, Pp->ga, q, Pq->ga and mf1
new_od_table = project.new_network_table("FULL")
column = _worksheet.Column()
column.name = "p"
column.expression = "p"
column.decimals = 0
new_od_table.add_column(1, column)
column.name = "orig. district"
column.expression = "Pp->ga"
new_od_table.add_column(2, column)
column.name = "q"
column.expression = "q"
new_od_table.add_column(3, column)
column.name = "dest. district"
column.expression = "Pq->ga"
new_od_table.add_column(4, column)
column.name = "mf1"
column.expression = "mf1"
new_od_table.add_column(5, column)

Then we summarize the data to get the sum of the demand matrix (`mf1`) by origin and destination district.

In [ ]:
# get data to summarize the table by district and close the network table
od_mf1_data = new_od_table.get_data()
d2d_summary = od_mf1_data.summarize(['orig. district', 'dest. district'], [['mf1', 'SUM']])
d2d_mf1 = project_table_db.create_table("d2d_mf1", d2d_summary, overwrite=True)
new_od_table.close()

In [ ]:
_pd.options.display.max_rows = 6
to_pandas(d2d_summary)

,orig. district,dest. district,Sum(mf1)
0,0,0,146
1,0,3,113
2,0,4,42
...,...,...,...
61,2,1,1934
62,2,6,1046
63,2,2,3086


A __join__ can be performed on an existing data table by specifying a second table to join it with the names of the column names to be joined upon. The method takes three arguments:
* __`att_name`__  is the column/attribute name of the current table object (the "inner table") for which you would like to perform the join on.
* __`right_table`__ is a data table object representing the second table to be joined with the current table.
* __`right_att_name`__ is the name of the corresponding column/attribute of the second table to perform the join on.

In [ ]:
# join on destination district
district_names = project_table_db.table('district names')
d2d_mf1_data = district_names.join('district', d2d_mf1, 'dest. district')
to_pandas(d2d_mf1_data)

,district,name,orig. district,dest. district,Sum(mf1)
0,0,CBD,0,0,146
1,0,CBD,1,0,1386
2,0,CBD,2,0,1684
...,...,...,...,...,...
61,9,External,5,9,142
62,9,External,6,9,127
63,9,External,9,9,125


The resulting data table will have a new column with the name of the destination district. 
We can delete the redundant column `'district'` and rename the column `'name'` to `'dest. district name'`. 
Then we will save the resulting data table under the same name.

In [ ]:
d2d_mf1_data.delete_attribute("district")
d2d_mf1_data.rename_attribute("name", "dest. district name")
d2d_mf1 = project_table_db.create_table("d2d_mf1", d2d_mf1_data, overwrite=True)

Now we will repeat the process, this time for the origin district.

In [ ]:
# join on origin district
district_names = project_table_db.table('district names')
d2d_mf1_data = district_names.join('district', d2d_mf1, 'orig. district')
d2d_mf1_data.delete_attribute("district")
d2d_mf1_data.rename_attribute("name", "orig. district name")
d2d_mf1 = project_table_db.create_table("d2d_mf1", d2d_mf1_data, overwrite=True)
to_pandas(d2d_mf1_data)

,orig. district name,dest. district name,orig. district,dest. district,Sum(mf1)
0,CBD,CBD,0,0,146
1,CBD,Central,0,1,191
2,CBD,External,0,9,34
...,...,...,...,...,...
61,External,North West,9,2,77
62,External,South East,9,5,29
63,External,South West,9,6,136


## Exporting from data tables

To export the contents of the data table, use the __`export_to_csv`__ method.
The code below gets the data of the `'nodes'` table and then exports it to a CSV file using ',' as the delimiter (the default delimiter is defined in the _Field separator_ entry in the Worksheet project settings):

In [ ]:
# export data table data to csv file
nodes_data = nodes_dt.get_data()
node_filepath = os.path.join(project_path, "Media/nodes.csv").replace("\\", "/")
nodes_data.export_to_csv(node_filepath, separator=",")

Let's display the first line of the resulting file:

In [ ]:
nodes = _pd.read_csv(node_filepath, "nodes.csv", engine='python')
nodes

,"Node,X-coord,Y-coord"
0,"1,633948.0000000000000000,5528783.000000000000..."
1,"2,633512.0000000000000000,5528805.000000000000..."
2,"3,633787.0000000000000000,5527975.000000000000..."
...,...
1047,"9436,633265.0000000000000000,5525614.000000000..."
1048,"9437,633195.0000000000000000,5525262.000000000..."
1049,"9438,632913.0000000000000000,5524506.000000000..."


## Performing a spatial join operation

A _Spatial Join_ operation affixes data from one data table to another based on its spatial relationship.

Spatial joins can be performed on geometry columns in a data table using the `Table.`__`spatial_join()`__ method. This method takes two arguments:
* __`right_table`__, which specifies the second data table object to be joined.
* __`predicate`__, which is a string dictating the type of spatial join to be performed.

The `predicate` argument must be one of `'INTERSECT'`, `'CONTAIN'`, or `'EQUALS'`.

In the code below, we use __`spatial_join`__ to establish correspondence between Emme network elements and zone polygons:

In [ ]:
# Spatial join operation
spatial_join_data = project_table_db.table("winnipeg zones").spatial_join(nodes_dt, 'CONTAIN')
to_pandas(spatial_join_data)

,ZONEID,DISTRICT,EMPLOYMENT,ZONETYPE,DESCRIPTIO,geometry,Node,X-coord,Y-coord,geometry:1
0,21,External,high,external,,"POLYGON((640481 5537272.070889, 640484.381886 ...",21,640481,5537046,POINT(640481 5537046)
1,42,South East,low,residential,,"POLYGON((634294.348449 5522172.892743, 634205....",42,635317,5522002,POINT(635317 5522002)
2,42,South East,low,residential,,"POLYGON((634294.348449 5522172.892743, 634205....",338,635717,5522754,POINT(635717 5522754)
...,...,...,...,...,...,...,...,...,...,...
956,83,Central,low,residential,,"POLYGON((630207.764608 5527169.205387, 630177....",605,629701,5527011,POINT(629701 5527011)
957,83,Central,low,residential,,"POLYGON((630207.764608 5527169.205387, 630177....",606,629567,5526976,POINT(629567 5526976)
958,83,Central,low,residential,,"POLYGON((630207.764608 5527169.205387, 630177....",607,629214,5526863,POINT(629214 5526863)


Note that:
- Each row now contains a node id column showing the nodes contained by each zone ZONEID.
- There are 959 resulting rows, whereas there are only 154 rows in the 'winnipeg zones' table.
- There is a second geometry column with type POINT created from the spatial join. It is not possible to save a data table with two geometry columns, so we will delete the second POINT geometry column labeled 'geometry:1'.

We are now ready to save the result to the new data table `nodes_in_zones`.

In [ ]:
spatial_join_data.delete_attribute('geometry:1')
project_table_db.create_table("nodes_in_zones", spatial_join_data, overwrite=True)

Table(nodes_in_zones)

## Performing a buffer (spatial) operation

A buffer operation produces a region of specified distance around the input geometry. Buffer operations can only be performed on data tables with a geometry column. Buffer distance is specified in the units of the data table spatial reference.

In the code below, the `Table.`__`buffer()`__ method is used to create a buffer on the transit stops to show catchment area.

We will start by creating a new network table of nodes with columns i (node ID) and attribute `ca_nbstops_i` (is transit stop indicator) and we will restrict the table to filter only those nodes where there are transit stops. Then we will save the node table as a data table object called 'stops'. Finally we will use the __buffer__ function and set the distance to 500.
The new list of buffered objects are then saved to a new 'stops_500m' table:

In [ ]:
new_stop_table = project.new_network_table("NODE")
new_stop_table.par("Filter").set("ca_nbstops_i>0")
column = _worksheet.Column()
column.name = "i"
column.expression = "i"
column.decimals = 0
new_stop_table.add_column(1, column)
column.name = "ca_nbstops_i"
column.expression = "ca_nbstops_i"
new_stop_table.add_column(2, column)
stops_dt = new_stop_table.save_as_data_table("stops", overwrite=True)
new_stop_table.close()

In [ ]:
# buffer
stops_with_buffer = stops_dt.buffer(500)
project_table_db.create_table("stops_500m", stops_with_buffer, overwrite=True)

Table(stops_500m)

## Mapping a data table

Spatial data tables contained within the project table database can be output as a GIS vector layer in a graphical worksheet. In the example below, we open a new worksheet, and add the spatial data from the project data tables to a new GIS vector layer. The GIS vector layer uses the information contained in the node buffer we created above:

In [ ]:
new_wk_i = root_ws_f.find_item(['New worksheet'])
new_wk = new_wk_i.open()
gis_layer = new_wk.add_layer_front("GIS vector")
gis_layer.load_data_tables()
gis_layer.par("Dataset").set("stops_500m")

In [ ]:
new_wk.close()

##Performing SQL Queries

As an alternative to using the methods presented above, the data tables contained within a table database can be accessed and manipulated with SQL commands using the __`query()`__ method. In the example below, the `'Node'`, `'ZONEID'`, `'DISTRICT'` and `'geometry'` columns are accessed from the 'nodes_in_zones' table 
(created in the section <a href="#Performing-a-spatial-join-operation">Performing a spatial join operation</a>)
with the resulting output grouped by `'Node'`:

In [ ]:
data_from_sql_query = project_table_db.query(
    "SELECT Node, ZONEID, DISTRICT, geometry \
    FROM nodes_in_zones GROUP BY Node;")

We can then save the results of our SQL query to the 'nodes_zones_districts' table:

In [ ]:
project_table_db.create_table("nodes_zones_districts", data_from_sql_query, overwrite=True)

Table(nodes_zones_districts)

In [ ]:
desktop.refresh_data()

## Creating a new data table database

In order to create a new table database, we will call a new instance of the __`Database()`__ class. `Database` accepts four arguments:
* __`db_mode`__ specifies the type of database being created. It can be either `'REGULAR'` or `'SPATIAL'`.
* __`filepath`__ is the location to save or access the database file.
* __`create`__ (optional) is a boolean flag which must be set to True if the database does not yet exist in the directory (ie, a new database will be created).
* __`overwrite`__ (optional) creates the new database even if it already exists in the directory (the file will be overwritten with a blank database).

The code below will create a new spatial database named `'test_database.db'` in the current project directory, and overwrite the file if a database of that name already exists:

In [ ]:
table_db_file = os.path.join(project_path, "test_database.db")
my_table_db = _dt.Database("SPATIAL", table_db_file, create=True, overwrite=True)

We can now create new tables in the table database using the __`create_table()`__ method.

The following will get data from the _Zones in partitions_ table and then save the result as a new data table.

In [ ]:
# open a network table and get it's data
table_item = root_ws_f.find_item(["General", "Matrices", "Zones in partitions"])
zone_table = table_item.open()
zone_table_data = zone_table.get_data()
# add table and it's data to a another table database
my_table_db.create_table("Winnipeg partitions", zone_table_data)
#zone_table.close()

Table(Winnipeg partitions)

Similarly, it is possible to copy a data table from one data table database to another.
Here we copy the data table `'nodes_in_zones'` from the projet data table database to the new `'test_database.db'`:

In [ ]:
nodes_in_zones = project_table_db.table("nodes_in_zones")
nodes_in_zones_data = nodes_in_zones.get_data()
my_table_db.create_table("Nodes in zones", nodes_in_zones_data)
#my_table_db.close()

Table(Nodes in zones)

We can remove tables from a database using __`delete_table()`__.
Here we will remove the data table `'nodes_in_zones'` from the projet data table database:

In [ ]:
project_table_db.delete_table("nodes_in_zones")

##Conclusion

This concludes the Data Table API Guide. For a detailed breakdown of all available functions and their signatures, please consult the Data Table API section in the Emme API Reference.